In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../")

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [ ]:
from copy import deepcopy

import torch
import torch.nn as nn
from transformers import T5ForConditionalGeneration, T5Tokenizer

# from final_solution.utils import get_inference_model


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# baseline_model = get_inference_model('../pretrained-rut5-2-fp16')
# baseline_model = get_inference_model("t5-small")
baseline_model = T5ForConditionalGeneration.from_pretrained(
    "t5-small", return_dict=True, torch_dtype=torch.float16
)
baseline_model.cuda()

student_model = deepcopy(baseline_model)

keep_encoder_blocks = [1, 2, 4, -1]
keep_decoder_blocks = [1, 2, 4, -1]

student_model.encoder.block = nn.ModuleList(
    [student_model.encoder.block[i] for i in keep_encoder_blocks]
)
student_model.decoder.block = nn.ModuleList(
    [student_model.decoder.block[i] for i in keep_decoder_blocks]
)
student_model.cuda();

In [ ]:
count_parameters(baseline_model) / count_parameters(student_model)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# [pin]
file_path = "../data/data-hard.csv"
root_path = "../data/"


df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)
df.sample(20)

,input_text,target_text,prefix
6999,🟢 Новости к этому часу ⚪️ Цена барреля Юралз ...,99-2,clsorg
2584,Конференция Смартлаба 24 июня в Санкт-Петербур...,53-3,clsorg
3867,​​🔍 Что шортят и покупают с плечом клиенты БКС...,99-2;160-3;227-3,clsorg
4711,🇷🇺#ALRS АЛРОСА оценивает capex рудника Мир в ...,4-3,clsorg
1829,$NVTK Завод Новатэк. Взрыв. ❗️Украинские СМИ ...,115-2,clsorg
4003,​​🟢 ИТОГИ ДНЯ. Российский рынок акций снова вы...,44-3;74-3;185-3,clsorg
3502,​Итоги последнего торгового дня мая на Московк...,111-3,clsorg
4198,⚠️🇷🇺#FEES #суд Инвесторы предъявили «Россетям»...,186-2,clsorg
3032,Результаты Globaltrans в 2022: рекордные показ...,220-3,clsorg
1871,6 апреля крупный онлайн-ретейлер Ozon опублико...,230-4,clsorg


In [ ]:
# m_name = "t5-small"
m_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

In [ ]:
from src.t5.dataset import NERDataModel

BATCH_SIZE = 128
EPOCHS = 10
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
data_module = NERDataModel(train_df, test_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [ ]:
import time

import torch

from final_solution.utils import generate_answer_batched

In [ ]:
# [pin]
baseline_model.eval()
baseline_model.cuda()
with torch.inference_mode(), torch.cuda.amp.autocast():
    begin = time.perf_counter()
    predictions = generate_answer_batched(
        trained_model=baseline_model,
        tokenizer=tokenizer,
        data=test_df,
        batch_size=64,
        max_target_length=40,
        num_beams=1,
    )
    end = time.perf_counter()
print(end - begin)

  0%|          | 0/29 [00:00<?, ?it/s]/home/worker/workspace/hakaton-gagarin-sentiment_interface/.conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|██████████| 29/29 [00:21<00:00,  1.38it/s]

21.01047930889763


In [ ]:
# [pin]
student_model.eval()
student_model.cuda()
with torch.inference_mode(), torch.cuda.amp.autocast():
    begin = time.perf_counter()
    predictions = generate_answer_batched(
        trained_model=student_model,
        tokenizer=tokenizer,
        data=test_df,
        batch_size=64,
        max_target_length=40,
        num_beams=1,
    )
    end = time.perf_counter()
print(end - begin)

100%|██████████| 29/29 [00:14<00:00,  1.95it/s]

14.869698968017474


Достигнуто ускорение в 1.4 раза при уменьшении числа параметров в 1.3 раза

Дальше требуется дообучить модель, например, на исходную задачу + дистилляционный лосс по активациям nn.KLDivLoss